# Notebook 02: Data Augmentation (Negative Sampling)

**Mục tiêu:** Sinh dữ liệu huấn luyện cho model Machine Learning (Pairwise Ranking).

Để train model, ta cần tạo ra các cặp dữ liệu `(Query, Candidate, Label)`:
* **Query:** Thông tin trích xuất từ BibTeX (đã clean).
* **Candidate:** Thông tin từ Ground Truth (Metadata).
* **Label:** 1 (Match) hoặc 0 (Non-match).

**Chiến lược sinh mẫu sai (Negative Sampling):**
1.  **Local Negatives (Hard):** Chọn metadata sai nằm *trong cùng một bài báo* (`paper_id` giống nhau). Ví dụ: BibTeX ref #1 ghép với GT ref #2 của cùng bài báo đó. Đây là mẫu khó vì cùng chủ đề.
2.  **Global Negatives (Easy):** Chọn metadata sai từ *bất kỳ bài báo nào khác*. Giúp model học phân biệt sự khác biệt rõ rệt (khác năm, khác tác giả...).



In [25]:

import pandas as pd
import numpy as np
import random
import os
from tqdm import tqdm

# --- CẤU HÌNH ---
INPUT_FILE = '../../dataset_final/clean_data/cleaned_data.pkl' # Load file PKL từ NB01
OUTPUT_FILE = '../../dataset_final/clean_data/train_augmented.pkl' # Lưu dạng PKL cho nhanh

# Tỷ lệ Negative/Positive
# 1 Positive sẽ đi kèm với bao nhiêu Negative?
NUM_NEGATIVES = 4  
# Trong đó bao nhiêu là Hard Negative (cùng bài báo)?
NUM_HARD_NEGATIVES = 2 
# Số còn lại sẽ là Easy Negative (random bài khác)

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)



## 1. Load dữ liệu sạch (Cleaned Data)
Ta load file `.pkl` được tạo ra từ Notebook 01 để tận dụng các trường `clean_title`, `clean_authors`, `clean_id`.


In [26]:

if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"❌ Không tìm thấy file: {INPUT_FILE}. Hãy chạy Notebook 01 trước.")

df_all = pd.read_pickle(INPUT_FILE)

# Chỉ lấy tập Train để Augment
df_train_src = df_all[df_all['partition'] == 'train'].copy()

print(f"Tổng số mẫu gốc (Partition = Train): {len(df_train_src)}")
print("Các cột có sẵn:", df_train_src.columns.tolist())


Tổng số mẫu gốc (Partition = Train): 6720
Các cột có sẵn: ['partition', 'source_type', 'key', 'paper_id', 'bib_content', 'gt_id', 'gt_title', 'gt_authors', 'clean_title', 'clean_authors', 'clean_id', 'clean_year', 'parse_method', 'gt_year']



## 2. Xây dựng Candidate Pools
Ta cần tạo kho chứa các ứng viên (Ground Truth) để lấy mẫu.


In [27]:
global_candidates = df_train_src[['gt_id', 'gt_title', 'gt_authors', 'gt_year', 'paper_id']].to_dict('records')

# 2. Local Pool: Gom nhóm theo Paper ID
local_pool = {}
for item in global_candidates:
    pid = item['paper_id']
    if pid not in local_pool:
        local_pool[pid] = []
    local_pool[pid].append(item)

print(f"Global Pool Size: {len(global_candidates)}")

Global Pool Size: 6720


## 3. Thực hiện Negative Sampling
Quy trình cho mỗi dòng BibTeX (Query):
1.  Tạo 1 cặp **Positive** (Chính nó).
2.  Tạo k cặp **Hard Negative** (Lấy từ `local_pool` của cùng `paper_id`).
3.  Tạo m cặp **Easy Negative** (Lấy ngẫu nhiên từ `global_candidates` khác `paper_id`).


In [28]:
augmented_rows = []

print("🚀 Đang sinh dữ liệu training (đã bao gồm Year)...")

for idx, row in tqdm(df_train_src.iterrows(), total=len(df_train_src)):
    # --- A. LẤY THÔNG TIN QUERY (BIBTEX) ---
    query_info = {
        'bib_title': row['clean_title'],
        'bib_authors': row['clean_authors'],
        'bib_id': row['clean_id'],
        'bib_year': row['clean_year'] # Năm trích xuất từ BibTeX
    }
    
    true_gt_id = row['gt_id']
    current_paper_id = row['paper_id']

    # --- B. TẠO POSITIVE SAMPLE (LABEL = 1) ---
    pos_row = query_info.copy()
    pos_row.update({
        'cand_id': row['gt_id'],
        'cand_title': row['gt_title'],
        'cand_authors': row['gt_authors'],
        'cand_year': row['gt_year'],  # <--- QUAN TRỌNG: Năm của Ground Truth
        'label': 1
    })
    augmented_rows.append(pos_row)

    # --- C. TẠO NEGATIVE SAMPLES (LABEL = 0) ---
    negatives_collected = 0
    
    # C.1: Hard Negatives (Local)
    local_candidates = local_pool.get(current_paper_id, [])
    valid_local_cands = [c for c in local_candidates if c['gt_id'] != true_gt_id]
    
    if valid_local_cands:
        k_hard = min(NUM_HARD_NEGATIVES, len(valid_local_cands))
        chosen_hard = random.sample(valid_local_cands, k_hard)
        
        for cand in chosen_hard:
            neg_row = query_info.copy()
            neg_row.update({
                'cand_id': cand['gt_id'],
                'cand_title': cand['gt_title'],
                'cand_authors': cand['gt_authors'],
                'cand_year': cand['gt_year'], # <--- Lấy năm của ứng viên sai
                'label': 0
            })
            augmented_rows.append(neg_row)
            negatives_collected += 1

    # C.2: Easy Negatives (Global)
    needed = NUM_NEGATIVES - negatives_collected
    attempts = 0
    while needed > 0 and attempts < 50:
        attempts += 1
        cand = random.choice(global_candidates)
        
        if cand['gt_id'] != true_gt_id and cand['paper_id'] != current_paper_id:
            neg_row = query_info.copy()
            neg_row.update({
                'cand_id': cand['gt_id'],
                'cand_title': cand['gt_title'],
                'cand_authors': cand['gt_authors'],
                'cand_year': cand['gt_year'], # <--- Lấy năm của ứng viên sai
                'label': 0
            })
            augmented_rows.append(neg_row)
            needed -= 1

🚀 Đang sinh dữ liệu training (đã bao gồm Year)...


100%|██████████| 6720/6720 [00:00<00:00, 26678.07it/s]


## 4. Lưu kết quả
Lưu thành DataFrame để dùng cho bước Feature Engineering.


In [29]:

df_augmented = pd.DataFrame(augmented_rows)

print("\n--- KẾT QUẢ DATA AUGMENTATION ---")
print(f"Số lượng mẫu ban đầu: {len(df_train_src)}")
print(f"Số lượng mẫu sau khi sinh: {len(df_augmented)}")
print(f"Tỷ lệ Positive/Negative:\n{df_augmented['label'].value_counts()}")

# Shuffle dữ liệu
df_augmented = df_augmented.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

# Lưu file PKL (Giữ nguyên kiểu dữ liệu List cho Authors)
df_augmented.to_pickle(OUTPUT_FILE)

print(f"\n✅ Đã lưu file dataset huấn luyện tại: {os.path.abspath(OUTPUT_FILE)}")
print("👉 BƯỚC TIẾP THEO: Chạy '03_feature_engineering.ipynb' để tạo vector đặc trưng từ file này.")



--- KẾT QUẢ DATA AUGMENTATION ---
Số lượng mẫu ban đầu: 6720
Số lượng mẫu sau khi sinh: 33600
Tỷ lệ Positive/Negative:
label
0    26880
1     6720
Name: count, dtype: int64

✅ Đã lưu file dataset huấn luyện tại: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\dataset_final\clean_data\train_augmented.pkl
👉 BƯỚC TIẾP THEO: Chạy '03_feature_engineering.ipynb' để tạo vector đặc trưng từ file này.


## 5. Kiểm tra mẫu dữ liệu (Sanity Check)
Kiểm tra xem mẫu Hard Negative trông như thế nào.


In [30]:

# %%
# Lấy 1 mẫu Positive
print("--- MẪU POSITIVE ---")
pos_sample = df_augmented[df_augmented['label'] == 1].iloc[0]
print(f"Bib Title:  {pos_sample['bib_title']}")
print(f"Cand Title: {pos_sample['cand_title']}")
print(f"Match ID:   {pos_sample['bib_id']} == {pos_sample['cand_id']}")

print("\n--- MẪU NEGATIVE (Check xem Title có khác nhau không) ---")
neg_sample = df_augmented[df_augmented['label'] == 0].iloc[0]
print(f"Bib Title:  {neg_sample['bib_title']}")
print(f"Cand Title: {neg_sample['cand_title']}")
print(f"Label:      {neg_sample['label']}")

--- MẪU POSITIVE ---
Bib Title:  Adding conditional control to text-to-image diffusion models
Cand Title: Adding Conditional Control to Text-to-Image Diffusion Models
Match ID:    == 2302-05543

--- MẪU NEGATIVE (Check xem Title có khác nhau không) ---
Bib Title:  Personalized federated learning via variational bayesian inference.
Cand Title: Narrow-Line Cooling and Imaging of Ytterbium Atoms in an Optical Tweezer Array.
Label:      0


In [ ]:
# lưu json để check

import json
with open('sample_positive.json', 'w', encoding='utf-8') as f:
    json.dump(pos_sample.to_dict(), f, ensure_ascii=False, indent=4)